In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from joblib import dump, load
import os

class ListingClassifier:
    def __init__(self, model_path='listing_classifier.joblib'):
        self.model_path = model_path
        self.model = None
        self.vectorizer = None
        
        # Load existing model if available
        if os.path.exists(model_path):
            self.load_model()
    
    def train(self, listings_data):
        # Create vectorizer and model
        self.vectorizer = TfidfVectorizer()
        self.model = LogisticRegression()
        
        # Prepare data
        texts = [text for text, _ in listings_data]
        labels = [label for _, label in listings_data]
        
        # Transform text data
        X = self.vectorizer.fit_transform(texts)
        X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2)
        
        # Train model
        self.model.fit(X_train, y_train)
        
        # Save model
        self.save_model()
        
        # Return accuracy score
        return self.model.score(X_test, y_test)
    
    def predict(self, description):
        if not self.model or not self.vectorizer:
            raise ValueError("Model not trained or loaded")
        
        # Transform text and predict
        X = self.vectorizer.transform([description])
        return self.model.predict(X)[0]
    
    def save_model(self):
        dump((self.model, self.vectorizer), self.model_path)
    
    def load_model(self):
        self.model, self.vectorizer = load(self.model_path)

if __name__ == "__main__":
    # Sample training data
    listings_data = [
        ("Complete game in excellent condition", 1),
        ("Like new, all components included", 1),
        ("Box only, no components", 0),
        ("Missing some cards but otherwise complete", 0),
        ("Brand new sealed game", 1),
        ("Replacement parts only", 0),
        ("All components present, great condition", 1),
        ("No rulebook or cards", 0),
        ("Components in mint condition", 1),
        ("Just the box and board", 0)
    ]
    
    # Initialize classifier
    classifier = ListingClassifier()
    
    # Train if no existing model
    if not os.path.exists(classifier.model_path):
        accuracy = classifier.train(listings_data)
        print(f"Model trained with accuracy: {accuracy:.2f}")
    
    # Test predictions
    test_descriptions = [
        "Complete game with all pieces",
        "Box only, no game included",
        "Brand new in shrink wrap"
    ]
    
    for desc in test_descriptions:
        prediction = classifier.predict(desc)
        print(f"\nDescription: {desc}")
        print(f"Prediction: {'Complete' if prediction == 1 else 'Incomplete'}")